<a href="https://colab.research.google.com/github/halen48/HarmonyShifter/blob/main/Harmony_Shifter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Harmony Shifter in Python</h1>
Author: <a href="https://github.com/halen48">Guilherme Novaes</a>

---
Refereces: 
*   <a href="https://hellomusictheory.com/learn/negative-harmony/">What Is Negative Harmony?</a>
*   <a href="https://en.wikipedia.org/wiki/Riemannian_theory">Riemannian theory</a>


In [ ]:
from google.colab import files
!pip install mido
from mido import MidiFile
import os

In [ ]:
ZIP_NAME = "shifted_harmonies.zip" #@param
#Clean the work dir
for f in os.listdir():
  if(os.path.isdir(f)):
    continue
  os.remove(f)

#1) Implementation

##Constants/Formulas

In [ ]:
SHARP = '♯'
FLAT = '♭'

<a href="https://www.inspiredacoustics.com/en/MIDI_note_numbers_and_center_frequencies">MIDI Notes</a>

In [ ]:
def mid2note(v):
  return ((v%12) ,v//12 - 1)

Channel 10 is always percurssion.<br>
In other words, the 10th channel is the percussion.<br>
Since first channel by mido is the 0 channel, the 10th channel is the number 9

In [ ]:
PERCUSSION_CHANNEL = 10

In [ ]:
def isPercussion(tracks):
  for m in tracks:
    dict_ = m.dict()
    if('channel' in dict_.keys()):
      return (dict_['channel'] == PERCUSSION_CHANNEL - 1)
  
  return False

##Chromatic Scale

In [ ]:
class ChromaticScale():
  def __init__(self, default_accidental):
    
    self.__ACCIDENTAL = ['sharp', 'flat']
    self.__default_accidental = default_accidental
    self.__default_accidental_index = self.__ACCIDENTAL.index(default_accidental)
    
    self._notes = {'sharp' : [], 'flat' : [], 'note2idx' : {}, 'idx2note' : {}}
    for i in range(ord('A'),ord('G')+1):
      self._notes['sharp'].append(chr(i))
      if(i != ord('B') and i!= ord('E')):
        self._notes['sharp'].append(chr(i)+SHARP)
    
    self._MAX_NOTES = len(self._notes['sharp'])

    self._notes['flat'] = [chr( ord('A') + (ord(v[0])- ord('A')+1)%7)+FLAT if len(v) > 1 else v for v in self._notes['sharp']]
    
    self.rotate('C',False)
  
    for i,note in enumerate(self._notes[default_accidental]):
      self._notes['note2idx'][note] = i
      self._notes['idx2note'][i] = note

    self.__circle5ths()

  def __circle5ths(self):
    self._circle = {'sharp' : [self._notes['sharp'][0]], 'flat' : [self._notes['flat'][0]], 'discrete' : [0], 'map' : {} }
    index = 7
    while index != 0:
      self._circle['sharp'].append(self._notes['sharp'][index])
      self._circle['flat'].append(self._notes['flat'][index])
      self._circle['discrete'].append(index)
      index = (index+7)%self._MAX_NOTES
    
    middle_slice = j = self._MAX_NOTES//2
    
    for i in range(1,middle_slice+1):
      note1, note2 = self.notes()[self._circle['discrete'][i]], self.notes()[self._circle['discrete'][j-middle_slice]]
      note1 = self._notes['note2idx'][note1] 
      note2 = self._notes['note2idx'][note2]
      
      self._circle['map'][note1] = note2
      self._circle['map'][note2] = note1
      j -= 1

  def rotate(self,start_note, do_circle = True):
    start_note = start_note.replace('#','♯').replace('b','♭')
    if(start_note in self._notes['sharp']):
      notes = self._notes['sharp']
    else:
      notes = self._notes['flat']
    while notes[0] != start_note:
      self._notes['sharp'].append(self._notes['sharp'].pop(0))
      self._notes['flat'].append(self._notes['flat'].pop(0))
    
    if(do_circle):
      self.__circle5ths()

  def print_circle(self, pettry = False):
    major = self._circle[self.__default_accidental].copy()
    self.rotate('A')
    minor = self._circle[self.__default_accidental].copy()
    self.rotate('C')
    ret = {'major':major, 'minor' : minor}
    if(pettry):
      return '\n'.join(['%s: %s'%(key,ret[key]) for key in ret])
    return ret
  
  def notes(self):
    return self._notes[self.__default_accidental]    
  
  def circle(self,key):
    return self._circle[key]

  def negativeMap(self):
    return '\n'.join(["%2s → %2s"%(self._notes['idx2note'][key],self._notes['idx2note'][self._circle['map'][key]]) for key in self._circle['discrete']])
    

##Implement your shift-harmonies formulas here

###All methods with the sufix below will be automatically included to perform the operation

In [ ]:
SUFIX = 'Harmony'

In [ ]:
class Harmonies(): 
  operations = {}

  def fifithHarmony(scale, note):
    v = mid2note(note)
    return scale.circle('discrete')[v[0]-1] + (12*(1+v[1]))

  def negativeHarmony(scale, note):
    v = mid2note(note)
    return scale.circle('map')[v[0]] + (12*(1+v[1]))
  
  def negative4thHarmony(scale,note):
    scale.rotate('F')
    ret = Harmonies.negativeHarmony(scale, note)
    scale.rotate('C')
    return ret


In [ ]:
for key in Harmonies.__dict__:
  if(SUFIX in key):
    Harmonies.operations[key.replace(SUFIX,'_%s')] = Harmonies.__dict__[key]

##Circle of fifiths


In [ ]:
#@title <img src="https://lh5.googleusercontent.com/jq0aEjROUrEuCw6aJS84iCzEd5jm707BRXFckMJ22VXm30HOHcP6vyKMEqPH4dVADi7FvOUu7a23C9PbH4aafuSbrdYRwlVCEAhscIGwbzxtSNFU5TDSairq_NlOvcH_H3bQSbXF" width="300"/>
scale = ChromaticScale('sharp')
print(scale.notes())
print(scale.print_circle(True))
print(scale.negativeMap())

['C', 'C♯', 'D', 'D♯', 'E', 'F', 'F♯', 'G', 'G♯', 'A', 'A♯', 'B']
major: ['C', 'G', 'D', 'A', 'E', 'B', 'F♯', 'C♯', 'G♯', 'D♯', 'A♯', 'F']
minor: ['A', 'E', 'B', 'F♯', 'C♯', 'G♯', 'D♯', 'A♯', 'F', 'C', 'G', 'D']
{7: 0, 0: 7, 2: 5, 5: 2, 9: 10, 10: 9, 4: 3, 3: 4, 11: 8, 8: 11, 6: 1, 1: 6}
 C →  G
 G →  C
 D →  F
 A → A♯
 E → D♯
 B → G♯
F♯ → C♯
C♯ → F♯
G♯ →  B
D♯ →  E
A♯ →  A
 F →  D


#2) Upload a MIDI

In [ ]:
#@title Optional (You can upload multiple)
uploaded = files.upload() 

Saving Gaynor_Gloria_I_Will_Survive_1.mid to Gaynor_Gloria_I_Will_Survive_1.mid


##Do the mathmagics

In [ ]:
FOLDER = "generated/"

In [ ]:
try:
  os.mkdir(FOLDER)
except FileExistsError:
  pass

In [ ]:
dir_files = os.listdir()
file_list_download = []
for name in Harmonies.operations:
  for file_name in dir_files:
    if(file_name[-4:] != '.mid'):
      continue
    tracks2conv = []
    mid = MidiFile(file_name, clip=True)
    for track in mid.tracks:
      if(not isPercussion(track)):
        tracks2conv.append(track)
    
    for v in tracks2conv:
      for p in v:
        try:
          p.note = Harmonies.operations[name](scale,p.note)
        except AttributeError:
          #Ignore the objects that isn't note messages in the midi
          pass
    
    file_list_download.append(FOLDER+(name%file_name))
    mid.save(file_list_download[-1])

#3) Download

In [ ]:
!zip -rm $ZIP_NAME $FOLDER
files.download(ZIP_NAME)

  adding: generated/ (stored 0%)
  adding: generated/fifith_Gaynor_Gloria_I_Will_Survive_1.mid (deflated 52%)
  adding: generated/negative_Gaynor_Gloria_I_Will_Survive_1.mid (deflated 52%)
  adding: generated/negative4th_Gaynor_Gloria_I_Will_Survive_1.mid (deflated 52%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>